# Проект по курсу "Рекомендательные системы"

## "Дружеское пари". Класс моделей: матричные факторизации
  
Правила заполнения ноутбуков на авто-проверку:
- повторить окружение преподавателя
```bash
pip install implicit==0.7.2 requests==2.32.3 rectools[lightfm]==0.12.0 pandas==2.2.3 numpy==1.26.4 scipy==1.12.0
```
- не добавлять новые импорты и не использовать дополнительные библиотеки. В противном случае ноутбук не пройдёт проверку и получит `0` баллов
- писать код только в ячейках с пометкой # YOUR CODE HERE, сразу после этой пометки
- не менять код преподавателя
- не добавлять новые ячейки
- следить, чтобы не было warning - они автоматом фейлят задание
- перед сдачей проверить, что весь ноутбук прогонятся от начала до конца и все тесты проходят

**В данном проекте вам нужно будет подбирать гипер-параметры моделей. Писать код для подбора гипер-параметров, использовать optuna и т.п. рекомендуем в отдельном ноутбуке.**

**Библиотеки implicit и lightfm не фиксируют random state при num_threads > 1. Если результат работы модели не сильно превышает необходимомый порог и рандом может опустить его ниже требуемого уровня, рекомендуем продолжить повышение качества модели: тюнинг гипер-параметров, подбор фичей, подбор метода обработки датасета**

## Импорты и данные

In [1]:
!python -V

Python 3.10.13


In [2]:
# Убедитесь, что вы не добавляете новые импорты в ноутбук. Решение должно быть ограничено данными библиотеками

import warnings
warnings.simplefilter("ignore")

import implicit
import rectools
import pandas as pd
import numpy as np
import scipy
import requests

print(implicit.__version__)
print(rectools.__version__)
print(pd.__version__)
print(np.__version__)
print(scipy.__version__)
print(requests.__version__)

0.7.2
0.12.0
2.2.3
1.26.4
1.12.0
2.32.3


In [3]:
import os.path

from implicit.als import AlternatingLeastSquares
from lightfm import LightFM

from rectools import Columns
from rectools.metrics import MAP, MeanInvUserFreq
from rectools.dataset import Dataset
from rectools.models import PureSVDModel, ImplicitALSWrapperModel, LightFMWrapperModel, model_from_config

# For implicit ALS
import os
import threadpoolctl
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

Если у вас нет данных, то используйте закомментированный код

In [5]:
# from tqdm.auto import tqdm
# import zipfile as zf

# url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

# req = requests.get(url, stream=True)

# with open('kion.zip', 'wb') as fd:
#     total_size_in_bytes = int(req.headers.get('Content-Length', 0))
#     progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
#     for chunk in req.iter_content(chunk_size=2 ** 20):
#         progress_bar.update(len(chunk))
#         fd.write(chunk)

# files = zf.ZipFile('kion.zip', 'r')
# files.extractall()
# files.close()

In [4]:
data_path = os.environ.get("DATA_PATH")
if data_path is None:
    data_path = "data_original"  # ваш путь к данным до папки data_original включительно (поменяйте при необходимости)

In [6]:
interactions = (
    pd.read_csv(os.path.join(data_path, "interactions.csv"), parse_dates=["last_watch_dt"])
    .rename(columns={'total_dur': Columns.Weight,
                     'last_watch_dt': Columns.Datetime})
)
users = pd.read_csv(os.path.join(data_path, "users.csv"))
items = pd.read_csv(os.path.join(data_path, "items.csv"))

print(interactions.shape)
interactions.head(5)

(5476251, 5)


,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [7]:
N_DAYS = 7

max_date = interactions['datetime'].max()
train = interactions[(interactions['datetime'] <= max_date - pd.Timedelta(days=N_DAYS))]
test = interactions[(interactions['datetime'] > max_date - pd.Timedelta(days=N_DAYS))]

catalog = train[Columns.Item].unique()

test_users = test[Columns.User].unique()
cold_users = set(test_users) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
hot_users = test[Columns.User].unique()
print(test.shape[0])
print(test[Columns.User].nunique())

306752
111240


## ImplicitALS (16 баллов)

### Ситуация:

Коллега вернулся из отпуска и вы вместе сели за улучшение модели. Внимательно изучив репозиторий библиотеки implicit вы увидели модель iALS и решаете попробовать ее в деле.

Чтобы работа была интереснее, вы заключаете пари с вашим коллегой о том, кто выбьет больше MAP@K на горячих пользователях. 

Правила пари: 
- Валидируемся на последней неделе (переменная `test`) и на горячих пользователях `hot_users`
- Можно собрать свой `Dataset` на основе `train`, трансформированного, если нужно
- Параметры модели задаются конфигом, которые будут передаваться в `model_from_config`

У вашего коллеги получилось выбить на ImplicitALS `MAP@K = 0.052`. Ваша задача побить его рекорд.

In [8]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    ### BEGIN SOLUTION
    train[Columns.Weight] = np.where(train['watched_pct'] > 10, 3, 1)
    dataset = Dataset.construct(train)
    ### END SOLUTION
    return dataset

config = {
    'cls': 'ImplicitALSWrapperModel',
    ### BEGIN SOLUTION
    'model': {
        'factors': 32,
        'alpha': 10,
        'regularization': 0.5,
        'num_threads': 16,
        'random_state': 23
    }
    ### END SOLUTION
}

In [9]:
%%time
model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.052

0.05425953080941801
CPU times: user 52.8 s, sys: 836 ms, total: 53.6 s
Wall time: 54.2 s


## SVD (16 баллов)

На ваш громкий спор с коллегой о том, что все дело в вашем удачном random seed, к вам подошел ваш лид. 

Узнав детали вашего спора, он дает вам комментарий, что iALS хороша, но погружение в матричную факторизацию следует начинать с `SVD`.

Вы переглянулись с коллегой и решаете уладить спор о random seed во втором раунде, используя новую модель.

Ваш коллега смогу выбить на SVD `MAP@K = 0.066`. Вы знаете, что делать.

In [13]:
def get_dataset(train: pd.DataFrame) -> Dataset:
    ### BEGIN SOLUTION
    train[Columns.Weight] = np.where(train['watched_pct'] > 10, 3, 1)
    dataset = Dataset.construct(train)
    ### END SOLUTION
    return dataset

config = {
    'cls': 'PureSVDModel',
    ### BEGIN SOLUTION
    'factors': 2,
    'random_state': 23,
    ### END SOLUTION
}

In [14]:
%%time

model = model_from_config(config)
dataset = get_dataset(train.copy())

assert config['cls'] == 'PureSVDModel'
assert dataset.item_features is None
assert dataset.user_features is None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)

print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.066

0.0715720372023016


## Dataset with features (8 баллов)

"Ну это ни в какие ворота!" - восклицает ваш коллега, увидев ваш победный конфиг. Из другого угла опенспейса доносится "А я говорил" от вашего лида.

В это время к вам сзади подходит продакт и интересуется предметом вашего спора.

Рассказав про особенности найденных вами моделей, он просит вас в них докинуть фичи, ведь на одних взаимодействиях далеко не уедешь.

Вы согласились, ведь это отличная возможность продолжить пари. Соберите `Dataset` с фичами по пользователям и итемам, который вы будете использовать дальше


In [20]:
def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    ### BEGIN SOLUTION
    users.fillna('Unknown', inplace=True)
    users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()
    user_features_frames = []
    for feature in ["sex", "age", "income"]:
        feature_frame = users.reindex(columns=[Columns.User, feature])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature
        user_features_frames.append(feature_frame)
    user_features = pd.concat(user_features_frames)
    
    items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
    items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
    genre_feature = items[["item_id", "genre"]].explode("genre")
    genre_feature.columns = ["id", "value"]
    genre_feature["feature"] = "genre"
    content_feature = items.reindex(columns=[Columns.Item, "content_type"])
    content_feature.columns = ["id", "value"]
    content_feature["feature"] = "content_type"
    item_features = pd.concat((genre_feature, content_feature))
    
    train[Columns.Weight] = np.where(train['watched_pct'] > 10, 3, 1)
    
    dataset = Dataset.construct(
        interactions_df=train,
        user_features_df=user_features,
        cat_user_features=["sex", "age", "income"],
        item_features_df=item_features,
        cat_item_features=["genre", "content_type"],
    )
    ### END SOLUTION
    return dataset

In [21]:
dataset_with_features = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert (dataset_with_features.user_features is not None) and (dataset_with_features.item_features is not None)

## ImplicitALS with features (20 баллов)

Собрав датасет с фичами, вы готовы к третьему раунду пари. 

Вы решаете начать снова с `iALS`, до сих пор удивляясь результатам модели `SVD`.

Ваш коллега изучил вашу технику подбора random seed и хитро улыбается вам.

Он смог выбить `MAP@K = 0.073`, теперь ваш ход.

In [22]:
config = {
    'cls': 'ImplicitALSWrapperModel',
    ### BEGIN SOLUTION
    'model': {
        'factors': 128,
        'alpha': 10,
        'regularization': 0.5,
        'num_threads': 12,
        'random_state': 23
    },
    'fit_features_together': True,
    ### END SOLUTION
}

In [23]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'ImplicitALSWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.073

0.0733584483390582
CPU times: user 9min 4s, sys: 17.9 s, total: 9min 22s
Wall time: 9min 51s


## LightFM with features (20 баллов)

И снова ор выше гор, ваш пайплайн подготовки датасета помог вам в очередной раз обойти вашего коллегу.

Не зная, к чему еще аппелировать, он зовет вашего лида, чтобы тот внимательно изучил полученные результаты.

"iALS с фичами это хорошо, но тут стоит попробовать факторизационные машины, попробуйте `LightFM`" - заключает он. Вы переключаетесь на изучение новой библиотеки, предвкушая финальный раунд.

Ваш коллега смог выжать из своего обновленного `Dataset` и `LightFM` скор `MAP@10 = 0.08`. Последний рывок.

In [30]:
def get_dataset_with_features(train: pd.DataFrame, users: pd.DataFrame, items: pd.DataFrame) -> Dataset:
    ### BEGIN SOLUTION
    users.fillna('Unknown', inplace=True)
    users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()
    user_features_frames = []
    for feature in ["sex", "age", "income"]:
        feature_frame = users.reindex(columns=[Columns.User, feature])
        feature_frame.columns = ["id", "value"]
        feature_frame["feature"] = feature
        user_features_frames.append(feature_frame)
    user_features = pd.concat(user_features_frames)
    
    items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
    item_features = items[["item_id", "content_type"]]
    item_features.rename(columns={"item_id": "id", "content_type": "value"}, inplace = True)
    item_features["feature"] = "content_type"
    
    train[Columns.Weight] = np.where(train['watched_pct'] > 10, 3, 1)
    
    dataset = Dataset.construct(
        interactions_df=train,
        user_features_df=user_features,
        cat_user_features=["sex", "age", "income"],
        item_features_df=item_features,
        cat_item_features=["content_type"],
    )
    ### END SOLUTION
    return dataset

config = {
    'cls': 'LightFMWrapperModel',
    ### BEGIN SOLUTION
    'model': {
        'loss': 'warp',
        'no_components': 12,
        'random_state': 23
    },
    'epochs': 10,
    ### END SOLUTION
}

In [32]:
%%time
model = model_from_config(config)
dataset = get_dataset_with_features(train.copy(), users.copy(), items.copy())

assert config['cls'] == 'LightFMWrapperModel'
assert dataset.item_features is not None
assert dataset.user_features is not None
raw_df = dataset.get_raw_interactions()
intersect = pd.merge(raw_df, test, on = Columns.UserItem)
assert intersect.shape[0] == 0
assert test.shape[0] == 306752
assert test[Columns.User].nunique() == 111240

model.fit(dataset)

recos = model.recommend(
    users=hot_users,
    dataset=dataset,
    k=10,
    filter_viewed=True,
)
print(MAP(10).calc(recos, test))

assert MAP(10).calc(recos, test) >= 0.08

0.08409101881695773
CPU times: user 52.6 s, sys: 1.31 s, total: 53.9 s
Wall time: 55.1 s
